<a href="https://colab.research.google.com/github/DanielDevito82/PhD-AI-Art-Histroy-Notebook/blob/main/TFRecord_Dataset_Multi_Label_Retrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepartions

## Imports


In [135]:
#Imports
import sys
import cv2
import os
import subprocess
import random as rn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import inspect
import tensorflow as tf
import IPython.display as display
from PIL import Image
import itertools
from tensorboard import notebook

if 'google.colab' in str(get_ipython()):
  # cv2.imshow is deactivated in GoogleLab
  from google.colab.patches import cv2_imshow

# To identify the platform. For the platform individual operation: MAC or Windows
import platform

# for the storing of logs, h5, files...
from datetime import datetime

# to evaluate the predicted classification
from sklearn.metrics import (classification_report,
                             mean_absolute_error,
                             mean_squared_error,
                             mean_squared_log_error,
                             multilabel_confusion_matrix)

from sklearn.metrics import (f1_score,
                             precision_score,
                             recall_score)

#Note Tensorflow includes the Framework Keras at Version 2.0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import Sequence, plot_model

# to force Tensporflow to use a single thread
from tensorflow.keras import backend # do I need this?

#EarlyStopping: Early stopping to track the val_loss and stop training once it begins increasing too much!
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [136]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
	return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _int64_feature_list(values):
	return tf.train.FeatureList(feature=[_int64_feature(v) for v in values])

### Global variables

In [140]:
# an CNN needs an uniform width,height,channels
g_Image_Shape = (128,128,1) # my shape
g_Image_Shape_ResNet = (224,224,3) # ResNet
g_Image_Shape_InceptionV3 = (299,299,3) # InceptionV3
g_batch_size = 1

if 'google.colab' in str(get_ipython()):
  g_File_training_csv = '/content/drive/MyDrive/Colab Notebooks/_training/features_train.csv'
  g_File_validation_csv = '/content/drive/MyDrive/Colab Notebooks/_validation/features_val.csv'
  g_Folder_training = '/content/drive/MyDrive/Colab Notebooks/_training'
  g_Folder_validation = '/content/drive/MyDrive/Colab Notebooks/_validation'
  path_model = '/content/drive/MyDrive/Colab Notebooks/Model/'
else:
  g_File_csv = 'Image/features.csv'
  g_Folder_Trimed = 'Images/_trimed/_1'
  path_model = 'Model/'

### Mount Google Drive


In [138]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Config the Hardware

In [139]:
print(tf.__version__)
print(platform.system())

physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
#try:
  #tf.config.experimental.set_memory_growth(physical_devices[0], True) # for a local enviroment, if your GPU memory should groth and not completely should be blocked. Because you need your GPu for other applications
#except:
  # Invalid device or cannot modify virtual devices once initialized.
#  pass

2.4.0
Linux
Num GPUs Available:  0


## Data Preprocessing


### Load csv

In [141]:
# Read the data (Image, Labels ...)
# Read the data (Image, Labels ...)
training_data = pd.read_csv(g_File_training_csv,sep=';').to_numpy()

# Read the data (Image, Labels ...)
validation_data = pd.read_csv(g_File_validation_csv,sep=';').to_numpy()
df = pd.read_csv(g_File_validation_csv,sep=';')
#np_array_with_imagesANDlabels = pd.read_csv(g_File_csv,sep=';').to_numpy()

In [142]:
# Extract the columns of the dataframe df for further preprocessing with the images
columns = []
for x  in df.columns:
    if x != 'image':
        columns.append(x)

# read how many layer / classes need to be defined in the last Dense Layer
lastLayer = len(columns)   

In [143]:
list_ds = tf.data.Dataset.list_files(str(g_Folder_training+'/*'), shuffle=False)
print(type(list_ds))
for f in list_ds.take(5):
  print(f.numpy())

<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>
b'/content/drive/MyDrive/Colab Notebooks/_training/0.jpg'
b'/content/drive/MyDrive/Colab Notebooks/_training/1.jpg'
b'/content/drive/MyDrive/Colab Notebooks/_training/1000.jpg'
b'/content/drive/MyDrive/Colab Notebooks/_training/1001.jpg'
b'/content/drive/MyDrive/Colab Notebooks/_training/1002.jpg'


In [144]:
def modifyImage(image):
  image = tf.image.resize(image,[299,299],method='nearest')
  #image = tf.cast(image, tf.float32) # führt zu Fehler
  image = tf.image.per_image_standardization(image)
  image = tf.image.flip_left_right(image)
  image = tf.image.flip_up_down(image)
  #image = tf.image.random_jpeg_quality(image, min_jpeg_quality=30, max_jpeg_quality=31, seed=None)
  return image

In [ ]:
# encoded_image = tf.io.read_file(g_Folder_training+'/'+'1000.jpg')
# img = tf.image.decode_jpeg(encoded_image, channels=3)
# img = tf.image.resize(img,[299,299],method='nearest')
# img = tf.image.per_image_standardization(img)
# #img = img / 255
# print(img)
# #img = modifyImage(img)
# plt.imshow(img,cmap='gray')
# plt.show()

### Read the images and write to TFRecord

In [145]:
# Write the raw image files to `images.tfrecords`.
# First, process the two images into `tf.train.Example` messages.
# Then, write to a `.tfrecords` file.
record_file = 'images.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
  for image in training_data:
    #print(g_Folder_Trimed+'/'+image[0])
    #with tf.io.gfile.GFile(g_Folder_training+'/'+image[0], 'rb') as f:
    #  encoded_image = f.read()
    encoded_image = tf.io.read_file(g_Folder_training+'/'+image[0])
    try:
      img = tf.image.decode_jpeg(encoded_image, channels=3)
      img = modifyImage(img)
      #height=img.shape[0]
      #width=img.shape[1]
      #channels=img.shape[2]
      #image_format=tf.compat.as_bytes('jpg')
      #filename=tf.compat.as_bytes(g_Folder_training+'/'+image[0])
      encoded_image = tf.image.encode_jpeg(img)
      labels = image[1:]
    except Exception:
      print("Skipping file with invalid JPEG data: {}".format(g_Folder_training+'/'+image[0]))
    else:
      context = tf.train.Features(feature={
        'image': _bytes_feature(encoded_image),
      })
      feature_lists = tf.train.FeatureLists(feature_list={
        "label": _int64_feature_list(labels),
      })

      # context = tf.train.Features(feature={
      #       'image/height': _int64_feature(height),
      #       'image/width': _int64_feature(width),
      #       'image/channels': _int64_feature(channels),
      #       'image/format': _bytes_feature(image_format),
      #       'image/filename': _bytes_feature(filename),
      #       'image/encoded': _bytes_feature(encoded_image),
      # })
      # feature_lists = tf.train.FeatureLists(feature_list={
      #   "image/labels": _int64_feature_list(labels),
      # })

      sequence_example = tf.train.SequenceExample(
        context=context,
        feature_lists=feature_lists)
      
      #print(str(example))

      writer.write(sequence_example.SerializeToString())

writer.close()

### Sandbox

In [146]:
#raw_image_dataset = tf.data.TFRecordDataset('images.tfrecords')
#print(raw_image_dataset)

# Example extraction for one image
for raw_record in raw_image_dataset.take(1):
  print(repr(raw_record))
  example = tf.train.SequenceExample()
  example.ParseFromString(raw_record.numpy())
  print(example)

<tf.Tensor: shape=(), dtype=string, numpy=b'\n\xc3\x82\x03\n\xbf\x82\x03\n\x05image\x12\xb4\x82\x03\n\xb0\x82\x03\n\xac\x82\x03\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\x01\x01\x02\x01\x01\x01\x02\x02\x02\x02\x02\x04\x03\x02\x02\x02\x02\x05\x04\x04\x03\x04\x06\x05\x06\x06\x06\x05\x06\x06\x06\x07\t\x08\x06\x07\t\x07\x06\x06\x08\x0b\x08\t\n\n\n\n\n\x06\x08\x0b\x0c\x0b\n\x0c\t\n\n\n\xff\xdb\x00C\x01\x02\x02\x02\x02\x02\x02\x05\x03\x03\x05\n\x07\x06\x07\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xff\xc0\x00\x11\x08\x01+\x01+\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x

### Load the TFRecord

In [ ]:
def create_dataset(path, repeat, batch):
  raw_image_dataset = tf.data.TFRecordDataset(path)

    # Create a dictionary for context and features to parses the sequence example into a tuple of dictionaries
  context = {
        'image': tf.io.FixedLenFeature([],dtype=tf.string),
  }
  # context = {
  #       'image/height': tf.io.FixedLenFeature([],dtype=tf.int64),
  #       'image/width': tf.io.FixedLenFeature([],dtype=tf.int64),
  #       'image/channels': tf.io.FixedLenFeature([],dtype=tf.int64),
  #       'image/format': tf.io.FixedLenFeature([],dtype=tf.string),
  #       'image/filename': tf.io.FixedLenFeature([],dtype=tf.string),
  #       'image/encoded': tf.io.FixedLenFeature([],dtype=tf.string),
  # }
  # feature_lists = {
  #   "image/labels": tf.io.FixedLenSequenceFeature((),dtype=tf.int64),
  # }
  feature_lists = {
    "label": tf.io.FixedLenSequenceFeature([],dtype=tf.int64),
  }
  # This op parses serialized sequence examples into a tuple of dictionaries
  def _parse_image_function(example_proto):
    # Parse the input tf.train.Example proto using the dictionary above.
    return tf.io.parse_sequence_example(example_proto, context_features=context, sequence_features=feature_lists)

  #raw_image_dataset.map
  parsed_image_dataset = raw_image_dataset.map(_parse_image_function, num_parallel_calls=8)
  
  # This dataset will go on forever
  parsed_image_dataset=parsed_image_dataset.repeat(repeat)

  # Set the number of datapoints you want to load and shuffle 
  #parsed_image_dataset = parsed_image_dataset.shuffle(SHUFFLE_BUFFER)

  # Set the batchsize
  parsed_image_dataset = parsed_image_dataset.batch(batch)

  # Create an iterator
  #iterator = iter(parsed_image_dataset)

  # Create your tf representation of the iterator
  #image_data = iterator.get_next()

  print('element_spec',parsed_image_dataset.element_spec)
  print(type(parsed_image_dataset))
  print(parsed_image_dataset)

  return parsed_image_dataset

In [ ]:
train_dataset = create_dataset('images.tfrecords', 1, 1)

element_spec ({'image': TensorSpec(shape=(None,), dtype=tf.string, name=None)}, {'label': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, {'label': TensorSpec(shape=(None,), dtype=tf.int64, name=None)})
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
<BatchDataset shapes: ({image: (None,)}, {label: (None, None)}, {label: (None,)}), types: ({image: tf.string}, {label: tf.int64}, {label: tf.int64})>


In [ ]:
# Create an iterator
iterator = iter(train_dataset)

# Create your tf representation of the iterator
image = iterator.get_next()

print(image[0])
print(image[0]['image'])
print(image[1])
print(image[2])

img=(image[0]['image'].numpy())
print(img)
display.Image(data=img)

{'image': <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\x01\x01\x02\x01\x01\x01\x02\x02\x02\x02\x02\x04\x03\x02\x02\x02\x02\x05\x04\x04\x03\x04\x06\x05\x06\x06\x06\x05\x06\x06\x06\x07\t\x08\x06\x07\t\x07\x06\x06\x08\x0b\x08\t\n\n\n\n\n\x06\x08\x0b\x0c\x0b\n\x0c\t\n\n\n\xff\xdb\x00C\x01\x02\x02\x02\x02\x02\x02\x05\x03\x03\x05\n\x07\x06\x07\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xff\xc0\x00\x11\x08\x01+\x01+\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x8

/usr/local/lib/python3.6/dist-packages/IPython/core/formatters.py:364: FormatterWarning: image/png formatter returned invalid type <class 'numpy.ndarray'> (expected (<class 'bytes'>, <class 'str'>)) for object: <IPython.core.display.Image object>
  FormatterWarning


<IPython.core.display.Image object>

In [ ]:
# for image_features in parsed_image_dataset.take(1):
#   print(repr(image_features))
#   image_raw = (image_features[0]['image/encoded'].numpy())
#   label = (image_features[1]['image/labels'])
#   tensor = image_features[0]['image/encoded']
#   print('Image_raw', image_raw)
#   print('Type',type(image_raw))
#   print('Label',label)
#   display.display(display.Image(data=image_raw))

## Monitoring

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d--%H%M")
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  notebook.list() # View open TensorBoard instances
  
  # Control TensorBoard display. If no port is provided, 
  # the most recently launched TensorBoard is used
  notebook.display(port=6006, height=1000)  
  # Load the TensorBoard notebook extension
  #%load_ext tensorboard

  %reload_ext tensorboard 
  %tensorboard --logdir logs

  log_directory = os.path.join("logs", timestamp)
else:
  print('Not running on CoLab')
  if platform.system() == 'Darwin' or platform.system() == 'Linux':
    # MACOS/LINUX: Use "logs\fit"
    log_directory = 'logs\fit'
  elif platform.system() == 'Windows':
    # WINDOWS: Use "logs\\fit"
    log_directory = 'logs\\fit'

  log_directory = log_directory + '\\' + timestamp


board = TensorBoard(log_dir=log_directory,
                    histogram_freq=1,
                    write_graph=True,
                    write_images=True,
                    update_freq='epoch',
                    profile_batch=1,
                    embeddings_freq=1)
  
 

## Create the model

In [ ]:
base_model = tf.keras.applications.InceptionV3(
                                  include_top=False, # the last layer is defined by classes and classifier_activation
                                  weights='imagenet',
                                  # input_tensor=None,
                                  input_shape=g_Image_Shape_InceptionV3 
                                  # pooling=None 
                                  #classes=lastLayer,
                                  #classifier_activation='sigmoid'
)
#print("Number of layers: {} ".format(len(original_model.layers)))
#print(type(base_model))
#print(base_model.summary())
base_model.trainable = False

#original_model_input  = tf.keras.Input(shape=g_Image_Shape_InceptionV3, name='image/encoded',tensor=parsed_image_dataset)
original_model_input  = tf.keras.Input(name='image/encoded',tensor=train_dataset)
#original_model_input.name='image/encoded'
#base_model.get_layer(index=0).input # keep the first layer
print(original_model_input)
layers = base_model(original_model_input, training=False) # keep the pre-last layer but drop the last layer 
layers = tf.keras.layers.GlobalAveragePooling2D()(layers)
original_model_output = Dense(lastLayer, activation='sigmoid')(layers)
model = Model(inputs=original_model_input, outputs=original_model_output)

# Flatten() is worser than GlobalAveragePooling https://stackoverflow.com/questions/49295311/what-is-the-difference-between-flatten-and-globalaveragepooling2d-in-keras

# learning_rate have to be slow because we are using a pretrained model
opt = tf.keras.optimizers.Adam(name = 'Adam')

# For a multi-label classification problems use binary_crossentropy
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['binary_accuracy','binary_crossentropy'])
model.summary()

AttributeError: ignored

Early stop

In [ ]:
parsed_image_dataset

<MapDataset shapes: ({image/channels: (), image/encoded: (), image/filename: (), image/format: (), image/height: (), image/width: ()}, {image/labels: (None,)}, {image/labels: ()}), types: ({image/channels: tf.int64, image/encoded: tf.string, image/filename: tf.string, image/format: tf.string, image/height: tf.int64, image/width: tf.int64}, {image/labels: tf.int64}, {image/labels: tf.int64})>

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, baseline=None, restore_best_weights=True)

In [ ]:
results = model.fit(parsed_image_dataset,
                   #epochs=15,
                   #validation_data=valid_batches_inception_v3,
                   #steps_per_epoch=step_size_train,
                   #callbacks=board,
                   #callbacks=early_stop,
                   #callbacks=[early_stop,board], # non Colab add board for TensorBoard
                   verbose=1
                   )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/functional.py:595: UserWarning: Input dict contained keys ['image/channels', 'image/filename', 'image/format', 'image/height', 'image/width'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


ValueError: ignored

# Training

In [ ]:
results = model.fit(train_batches_inception_v3,
                   epochs=15,
                   validation_data=valid_batches_inception_v3,
                   steps_per_epoch=step_size_train,
                   #callbacks=board,
                   #callbacks=early_stop,
                   callbacks=[early_stop,board], # non Colab add board for TensorBoard
                   verbose=1
                   )

# Evaluate

In [ ]:
model.evaluate(valid_batches_inception_v3)

In [ ]:
metrics =  pd.DataFrame(results.history)

Plot accurency

In [ ]:
# plot train and validation loss
plt.plot(metrics[['loss']])
plt.plot(metrics[['val_loss']])
plt.title('model train loss vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'validation val_loss'], loc='upper right')
plt.show()

Plot validation loss


In [ ]:
# plot train and validation loss
plt.plot(metrics[['binary_accuracy']])
plt.plot(metrics[['val_binary_accuracy']])
plt.title('model train validation vs validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train accuracy', 'validation val_accuracy'], loc='upper right')
plt.show()

# Finetuning


Set model to trainable

In [ ]:
base_model.trainable = True

# learning_rate have to be slow because we are using a pretrained model
opt = tf.keras.optimizers.Adam(learning_rate = 0.00001, name = 'Adam', clipvalue=1)

# For a multi-label classification problems use binary_crossentropy
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['binary_accuracy','binary_crossentropy'])
model.summary()

In [ ]:
results2 = model.fit(train_batches_inception_v3,
                   epochs=5,
                   validation_data=valid_batches_inception_v3,
                   steps_per_epoch=step_size_train,
                   #callbacks=board,
                   #callbacks=early_stop,
                   callbacks=[early_stop,board], # non Colab add board for TensorBoard
                   verbose=1
                   )

# Evaluate

In [ ]:
model.evaluate(valid_batches_inception_v3)

In [ ]:
metrics =  pd.DataFrame(results.history)

Plot accurency

In [ ]:
# plot train and validation loss
plt.plot(metrics[['loss']])
plt.plot(metrics[['val_loss']])
plt.title('model train loss vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'validation val_loss'], loc='upper right')
plt.show()

Plot validation loss

In [ ]:
# plot train and validation loss
plt.plot(metrics[['binary_accuracy']])
plt.plot(metrics[['val_binary_accuracy']])
plt.title('model train validation vs validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train accuracy', 'validation val_accuracy'], loc='upper right')
plt.show()

# Save the model

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model/2
model.save('/content/drive/MyDrive/Colab Notebooks/Model/saved_model/my_model/1') 